In [ ]:
!pip install openai

In [ ]:
api_key = "API_KEY_HERE"

In [ ]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import openai

# OpenAI API key
openai.api_key = api_key

In [ ]:
import pandas as pd
import ast
import nltk
from tqdm import tqdm
import time
# sleep import
from time import sleep

In [ ]:
# df = pd.read_csv('data_2015_to_2021.csv')
df = pd.read_csv('data_2014.csv')

In [ ]:
comments = df['comments'][20]
comments = ast.literal_eval(comments)

# each comment is a dictionary with keys: body and score
# sort comments by score descending
comments = sorted(comments, key=lambda x: x['score'], reverse=True)

# filter out comments that have no ? in body
comments = [comment for comment in comments if '?' in comment['body']]

# filter out comments that have more than 15 words or less than 5 words
comments = [comment for comment in comments if len(nltk.word_tokenize(comment['body'])) > 5 and len(nltk.word_tokenize(comment['body'])) < 50]

# filter out comments that have score less than 10
comments = [comment for comment in comments if comment['score'] > 10]

# filter out comments that have https in body
comments = [comment for comment in comments if 'https' not in comment['body']]

In [ ]:
print(time.time())
sleep(1)
print(time.time())

In [ ]:
# loop through each row in the dataframe
for index, row in df.iterrows():
    print(f'{index+1} row')
    comments = row['comments']
    comments = ast.literal_eval(comments)
    comments = sorted(comments, key=lambda x: x['score'], reverse=True)
    comments = [comment for comment in comments if '?' in comment['body']]
    comments = [comment for comment in comments if len(nltk.word_tokenize(comment['body'])) > 5 and len(nltk.word_tokenize(comment['body'])) < 50]
    comments = [comment for comment in comments if comment['score'] > 10]
    comments = [comment for comment in comments if 'http' not in comment['body']]
    for i, comment in enumerate(comments):
        query = f'Describe the topic of the question below in a single word/phrase. \n\nQ: {comment["body"]}\n\nA:'
        t = time.time()
        response = openai.Completion.create(
            model="text-davinci-003",
            prompt=query,
            max_tokens=5,
            temperature=0
        )
        time_left = 1.0 - (time.time() - t)
        if time_left > 0:
            sleep(time_left)
        topic = response['choices'][0]['text']
        # print(f'{i+1}. {topic}')
        comments[i]['topic'] = topic
    df.at[index, 'comments'] = comments
    df.to_csv('data_2014_with_topics_full.csv', index=False)